    Ben Christensen
    Math 347
    April 3, 2018

Code the Drazin Inverse and use it to predict links in a network

In [5]:
import numpy as np
from scipy import linalg as la
import csv

In [6]:
# Helper function for problems 1 and 2.
def index(A, tol=1e-5):
    """Compute the index of the matrix A.

    Parameters:
        A ((n,n) ndarray): An nxn matrix.

    Returns:
        k (int): The index of A.
    """

    # test for non-singularity
    if not np.allclose(la.det(A),0):
        return 0

    n = len(A)
    k = 1
    Ak = A.copy()
    while k <= n:
        r1 = np.linalg.matrix_rank(Ak)
        r2 = np.linalg.matrix_rank(np.dot(A,Ak))
        if r1 == r2:
            return k
        Ak = np.dot(A,Ak)
        k += 1

    return k


# Problem 1
def is_drazin(A, Ad, k):
    """Verify that a matrix Ad is the Drazin inverse of A.

    Parameters:
        A ((n,n) ndarray): An nxn matrix.
        Ad ((n,n) ndarray): A candidate for the Drazin inverse of A.
        k (int): The index of A.

    Returns:
        (bool) True of Ad is the Drazin inverse of A, False otherwise.
    """
    #Test each of the three cases
    if np.all(np.isclose(A@Ad, Ad@A)):
        #Compute A^k once
        B = np.linalg.matrix_power(A, k)
        if np.all(np.isclose(B@A@Ad, B)):
            if np.all(np.isclose(Ad@A@Ad, Ad)):
                return True
    #If the three properties don't hold, we return false
    return False



# Problem 2
def drazin_inverse(A, tol=1e-4):
    """Compute the Drazin inverse of A.

    Parameters:
        A ((n,n) ndarray): An nxn matrix.

    Returns:
       ((n,n) ndarray) The Drazin inverse of A.
    """
    #Implement Algorithm 15.1
    n = A.shape[0]
    f = lambda x: abs(x) > tol
    g = lambda x: abs(x) <= tol
    Q1, S, k1 = la.schur(A, sort=f)
    Q2, T, k2 = la.schur(A, sort=g)
    U = np.hstack((S[:,:k1], T[:,:n-k1]))
    #Calculate U^-1 only once
    U_inv = la.inv(U)
    V = U_inv@A@U
    Z = np.zeros((n,n))
    if k1 != 0:
        Z[:k1, :k1] = la.inv(V[:k1, :k1])
    return U@Z@U_inv


# Problem 3
def effective_resistance(A):
    """Compute the effective resistance for each node in a graph.

    Parameters:
        A ((n,n) ndarray): The adjacency matrix of an undirected graph.

    Returns:
        ((n,n) ndarray) The matrix where the ijth entry is the effective
        resistance from node i to node j.
    """
    n = A.shape[0]
    #Compute the Laplacian of A
    L = np.diag(np.sum(A, axis=0)) - A
    R = []
    #Iteravely find L~_j^D and compute the jth row of R
    for j in range(n):
        L_j = L.copy()
        temp = np.zeros(n)
        temp[j] = 1
        L_j[j] = temp
        L_D = drazin_inverse(L_j)
        R.append(list(np.diag(L_D)))

    R = np.array(R)
    #Make the diagonal entries zero
    R = R - np.diag(np.diag(R))
    return R



# Problems 4 and 5
class LinkPredictor:
    """Predict links between nodes of a network."""

    def __init__(self, filename='/Users/benchristensen/Desktop/ACME Python Labs/Volume1-Student-Materials/DrazinInverse/social_network.csv'):
        """Create the effective resistance matrix by constructing
        an adjacency matrix.

        Parameters:
            filename (str): The name of a file containing graph data.
        """
        node_names = []
        with open(filename, 'r') as networkfile:
            networkfile.readline()
            lines = [line for line in networkfile]

        for line in lines:
            name1, name2 = line.strip().split(',')
            node_names.append(name1)
            node_names.append(name2)
        #A list of all the names in the network
        node_names = set(node_names)
        n = len(node_names)
        #A dictionary of the names so each name has an index
        #for our adjacency matrix.
        node_names = list(node_names)
        name_dict = dict(zip(node_names, np.arange(n)))
        #Adjacency Matrix
        A = np.zeros((n,n))
        for line in lines:
            name1, name2 = line.strip().split(',')
            A[name_dict[name1], name_dict[name2]] = 1
        A = A + A.T
        self.names = node_names
        self.A = A
        self.R = effective_resistance(A)
        self.dict = name_dict


    def predict_link(self, node=None):
        """Predict the next link, either for the whole graph or for a
        particular node.

        Parameters:
            node (str): The name of a node in the network.

        Returns:
            node1, node2 (str): The names of the next nodes to be linked.
                Returned if node is None.
            node1 (str): The name of the next node to be linked to 'node'.
                Returned if node is not None.

        Raises:
            ValueError: If node is not in the graph.
        """
        #Find the two nodes if node is None
        if node is None:
            new = self.R*(self.A == 0)
            minval = np.min(new[new > 0])
            loc = np.where(new==minval)
            return self.names[loc[0][0]], self.names[loc[1][0]]
        #Find one node if node is not None
        else:
            #Raise error if not in the graph
            try:
                row = self.dict[node]
            except Exception as e:
                print("Node was not in the graph:", e)
            #Only check for nodes not already connected
            relevant = self.R[row]*(self.A[row] == 0)
            minval = np.min(relevant[relevant>0])
            loc = np.where(relevant==minval)

            return self.names[loc[0][0]]


    def add_link(self, node1, node2):
        """Add a link to the graph between node 1 and node 2 by updating the
        adjacency matrix and the effective resistance matrix.

        Parameters:
            node1 (str): The name of a node in the network.
            node2 (str): The name of a node in the network.

        Raises:
            ValueError: If either node1 or node2 is not in the graph.
        """
        try:
            name1 = self.dict[node1]
            name2 = self.dict[node2]
        except Exception as e:
            print("Node was not in the graph:", e)
        #Update the A and R matrices when adding a friend
        self.A[name1, name2] += 1
        self.A[name2, name1] += 1
        self.R = effective_resistance(self.A)


In [7]:
A = np.array([[1, 3, 0, 0],
              [0, 1, 3, 0],
              [0, 0, 1, 3],
              [0, 0, 0, 0]])
Ad = drazin_inverse(A)
k = 1
print(is_drazin(A, Ad, k))
B = np.array([[1, 1, 3],
              [5, 2, 6],
              [-2, -1, -3]])
Bd = np.zeros((3,3))
k = 3
print(is_drazin(B, Bd, k))

True
True


In [11]:
Predictor = LinkPredictor()
Predictor.predict_link()


('Emily', 'Oliver')

In [12]:
Predictor.predict_link("Melanie")

'Carol'

In [14]:
for i in range(3):
    new_friend = Predictor.predict_link("Alan")
    Predictor.add_link("Alan", new_friend)
    print(new_friend)


Sonia
Piers
Abigail
